<h3>Ubibot Sensor Data Monitoring</h3>

This notebook fetches sensor data from the Ubibot API and prints the latest values
for temperature, humidity, and voltage.

The script performs the following:
1. Configures the Ubibot API URL using the `CHANNEL_ID` and `API_KEY`.
2. Fetches sensor data from the API and extracts `Temperature`, `Humidity`, and `Voltage`.
3. Continuously monitors the sensor data at one-second intervals.

Make sure to replace `CHANNEL_ID` and `API_KEY` with your actual Ubibot credentials.

In [ ]:
!pip install requests

Initial configuration:

In [ ]:
import time
import json
import requests

# Configuration
CHANNEL_ID = 86828  # Replace with your Ubibot Channel ID
API_KEY = "608ecbcd79e64387b6e16f013c7fbe25"  # Replace with your Ubibot API Key
UBIBOT_URL = f"https://webapi.ubibot.com/channels/{CHANNEL_ID}?api_key={API_KEY}"

# Sensor mapping
sensor_names = {
    "field1": "Temperature",
    "field2": "Humidity",
    "field3": "Voltage"
}

Fetch Latest Sensor Data:

In [ ]:
def fetch_latest_sensor_data(url, sensor_names):
    """
    Fetches the latest sensor data from the Ubibot API.
    
    Args:
        url (str): Ubibot API endpoint.
        sensor_names (dict): Mapping of field names to sensor names.
    
    Returns:
        dict: Latest sensor values with corresponding sensor names.
    """
    try:
        response = requests.request("GET", url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            last_values_string = data['channel']['last_values']
            last_values = json.loads(last_values_string)
            
            sensor_data = {}
            for field, value in last_values.items():
                if field in sensor_names:
                    sensor_data[sensor_names[field]] = value['value']
            return sensor_data
        else:
            print(f"Failed to fetch data: {response.status_code}")
            print(f"Response: {response.text}")
            return None
    except requests.exceptions.Timeout as e:
        print(f"Request timed out: {e}")
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")
    except KeyError as e:
        print(f"Failed to extract data: {e}")
    except ValueError as e:
        print(f"Failed to extract data: {e}")
    return None

# Test the function
sensor_data = fetch_latest_sensor_data(UBIBOT_URL, sensor_names)
if sensor_data:
    print("Latest Sensor Data:")
    for sensor, value in sensor_data.items():
        print(f"{sensor}: {value}")

Continuous Monitoring:

In [ ]:
def monitor_sensors(url, sensor_names, interval=1):
    """
    Continuously monitors the sensor data at regular intervals.
    
    Args:
        url (str): Ubibot API endpoint.
        sensor_names (dict): Mapping of field names to sensor names.
        interval (int): Time interval (in seconds) between requests.
    """
    try:
        while True:
            sensor_data = fetch_latest_sensor_data(url, sensor_names)
            if sensor_data:
                print("Sensor Data at", time.ctime())
                for sensor, value in sensor_data.items():
                    print(f"{sensor}: {value}")
                print()
            time.sleep(interval)
    except KeyboardInterrupt:
        print("Monitoring interrupted by user.")

monitor_sensors(UBIBOT_URL, sensor_names)